In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder as LE

In [23]:
data=pd.read_csv("titanic_train.csv")

In [24]:
data=data.loc[:,['Survived', 'Pclass', 'Sex', 'Age', 'SibSp','Parch']]

In [25]:
X=data.loc[:,['Pclass', 'Sex', 'Age', 'SibSp','Parch']]

In [26]:
y=data.loc[:,'Survived']

In [35]:
le=LE()

In [38]:
X["Sex"]=le.fit_transform(X.Sex)

In [40]:
X.Age.fillna(np.mean(X.Age),inplace=True)

In [42]:
def gini_impurity(columns):
    val,counts=np.unique(columns,return_counts=True)
    gini_impurity=0
    for count in counts:
        p=count/len(columns)
        gini_impurity+=p*(1-p)
    return gini_impurity

In [92]:
def gini_gini(X,y,label):
    pivot= np.mean(X[label])
    y_left=y[X[label]<pivot]
    y_right=y[X[label]>=pivot]
    if len(y_left)==0 or len(y_right)==0:
        return -1000
    gini_gini=gini_impurity(y)-gini_impurity(y_left)-gini_impurity(y_right)
    
    return -gini_gini

In [93]:
gini_gini(X,y,"Sex")

0.21626576181475737

In [94]:
class Node:
    def __init__(self,Label=None,value=None,result=None):
        self.label=Label
        self.value=value
        self.result=result

In [108]:
class DeciTreeGini:
    def __init__(self,max_depth):
        self.max_depth=max_depth
        
    def fit(self,X,y):
        self.X=X
        self.y=y
        self.root=self.generator(self.X,self.y,self.max_depth)
    def generator(self,X,y,depth):
        
        if depth==1:
            return Node(result=np.mean(y))
        
        gains=[]
        for label in X.columns:
            gain=gini_gini(X,y,label)
            gains.append((gain,label))
            
        selected=min(gains)[1]
        pivot=np.mean(X[selected])
        
        X_right=X[X[selected]>=pivot]
        y_right=y[X[selected]>=pivot]
        X_left=X[X[selected]<pivot]
        y_left=y[X[selected]<pivot]
        
        if (len(y_left)==0) or (len(y_right)==0):
            return Node(result=np.mean(y))
        
        node=Node(selected,pivot)
        node.left=self.generator(X_left,y_left,depth-1)
        node.right=self.generator(X_right,y_right,depth-1)
        
        return node
    
    def display(self,node,indent=""):
        if node.label==None:
            if node.result<0.5:
                print(indent,"Died")
            else:
                print(indent,"Survived")
            return
                
        print(indent,node.label,node.value)
            
        self.display(node.left,indent+"\t")
        self.display(node.right,indent+"\t")
            
    def predict_one(self,item,node):
        if node.label==None:
            return node.result
        pivot=node.value
        if(item[node.label]<pivot):
            return self.predict_one(item,node.left)
        else:
            return self.predict_one(item,node.right)
        
    def predict(self,X):
        y=[]
        for row in X.iterrows():
            results=self.predict_one(row[1],self.root)
            y.append(int(results>=0.5))
        return np.array(y)
        
         
    def score(self,X,y):
        yp=self.predict(X)
        return sum(y.values==yp)/len(y)
        

In [109]:
model=DeciTreeGini(5)

In [110]:
model.fit(X,y)

In [111]:
model.display(model.root)

 Sex 0.6475869809203143
	 Survived
	 Died


In [112]:
model.score(X,y)

0.7867564534231201

In [2]:
class Node:
    def __init__(self,label=None,value=None,result=None):
        self.label=label
        self.value=value
        self.result=result

In [ ]:
class DT:
    
    def __init__(self, max_depth=5):
        self.max_depth=max_depth
        
    def fit(self,X,y):
        self.X=X
        self.y=y
        self.root=self.generate(self.X,self.y,self.max_depth)
        
    def generate(self,X,y,depth):
        if depth==1:
            return Node(result=np.mean(y))
        
        gains=[]
        for column in X.columns:
            gain=info_gain(column)
            gains.append(gain,)